In [2]:
import numpy as np
import pandas as pd

In [33]:
def BrownRobinson(A, max_iter=1000, eps=1e-3, seed=None):
    assert len(A.shape) == 2
    N = A.shape[0]
    M = A.shape[1]

    rng = np.random.default_rng(seed)

    i = rng.integers(low=0, high=N-1)
    A_cumsum = np.zeros(N)
    B_cumsum = np.zeros(M)
    v_hat = np.inf

    table_data = []
    for k in range(1, max_iter+1):
        table_row = []
        table_row.append(i+1)

        B_cumsum = B_cumsum + np.squeeze(A[i, :])
        j = rng.choice(np.flatnonzero(B_cumsum == B_cumsum.min()))
        table_row.extend(np.round(B_cumsum, 1))
        v_lower = B_cumsum[j] / k

        table_row.append(j+1)

        A_cumsum = A_cumsum + np.squeeze(A[:, j])
        i = rng.choice(np.flatnonzero(A_cumsum == A_cumsum.max()))
        table_row.extend(np.round(A_cumsum, 1))
        v_upper = A_cumsum[i] / k

        v_hat_new = ( v_lower + v_upper ) / 2
        table_row.extend(np.round([v_lower, v_hat_new, v_upper], 2))

        table_data.append(table_row)
        if abs(v_hat_new - v_hat) <= eps:
            break
        v_hat = v_hat_new

    # generate df
    A_labels = [ f'A_{i}' for i in range(1, N+1) ] 
    B_labels = [ f'B_{i}' for i in range(1, M+1) ] 
    v_labels = ['v_lower', 'v_hat', 'v_upper']
    df_table = pd.DataFrame(table_data,
                            columns=['i', *B_labels, 'j', *A_labels, *v_labels])
    df_table.index += df_table.index + 1

    return df_table

In [34]:
A = np.array([
    [7, 2, 9],
    [2, 9, 0],
    [9, 0, 11]
    ])

In [59]:
my_seed = 2024
br_table = BrownRobinson(A, seed=my_seed)
br_table.head()

,i,B_1,B_2,B_3,j,A_1,A_2,A_3,v_lower,v_hat,v_upper
1,1,7.0,2.0,9.0,2,2.0,9.0,0.0,2.00,5.50,9.0
3,2,9.0,11.0,9.0,3,11.0,9.0,11.0,4.50,5.00,5.5
5,1,16.0,13.0,18.0,2,13.0,18.0,11.0,4.33,5.17,6.0
7,2,18.0,22.0,18.0,1,20.0,20.0,20.0,4.50,4.75,5.0
9,1,25.0,24.0,27.0,2,22.0,29.0,20.0,4.80,5.30,5.8


In [60]:
N, M = 4, 5
low, high = 50, 500
A = np.random.randint(low, high, ( N, M ))
display(pd.DataFrame(A))

,0,1,2,3,4
0,235,290,225,96,55
1,82,274,64,98,381
2,404,267,227,349,335
3,272,165,149,53,469


In [7]:
br_table = BrownRobinson(A, seed=my_seed)
display(br_table.tail())

,i,B_1,B_2,B_3,B_4,B_5,j,A_1,A_2,A_3,A_4,v_lower,v_hat,v_upper
1591,3,218200.0,217990.0,288763.0,218126.0,269777.0,2,215177.0,207919.0,218515.0,218343.0,273.86,274.19,274.52
1593,3,218371.0,218222.0,289141.0,218490.0,270086.0,2,215523.0,208073.0,218747.0,218674.0,273.80,274.13,274.46
1595,3,218542.0,218454.0,289519.0,218854.0,270395.0,2,215869.0,208227.0,218979.0,219005.0,273.75,274.10,274.44
1597,4,219040.0,218785.0,289832.0,218974.0,270801.0,2,216215.0,208381.0,219211.0,219336.0,273.82,274.17,274.51
1599,4,219538.0,219116.0,290145.0,219094.0,271207.0,4,216481.0,208763.0,219575.0,219456.0,273.87,274.17,274.47
